In [2]:
import pandas as pd
import re

In [88]:

pd.set_option('display.max_columns', None)

data = pd.read_csv('trail1.csv')

data.sample(10)

,Video Title,Video URL,Youtube Creator,YT Creator URL,Is Ad,Time
1587,Yor pays for Pizza #anime #spyxfamily #yorforg...,https://www.youtube.com/watch?v=j9vJt3llrHE,KingChris,https://www.youtube.com/channel/UCu-pLexbz0lzA...,False,"12 Mar 2023, 17:07:58 IST"
2153,[Free] Building + Hosting React App For free o...,https://www.youtube.com/watch?v=Fi75tq9JikI,CodeWithHarry,https://www.youtube.com/channel/UCeVMnSShP_Ivi...,False,"6 Jan 2023, 10:49:18 IST"
11846,Surjective (onto) and injective (one-to-one) f...,https://www.youtube.com/watch?v=xKNX8BUWR0g,Khan Academy,https://www.youtube.com/channel/UC4a-Gbdw7vOac...,False,"27 Aug 2021, 10:23:35 IST"
3280,Drawing Yor Forger in Spy x Family #yorforger ...,https://www.youtube.com/watch?v=o_vL7Dtpywc,Draw so easy Anime,https://www.youtube.com/channel/UCNPAJgjON65Fa...,False,"4 Nov 2022, 23:50:40 IST"
12461,Haan Main Jitni Martaba | All The Best | Ajay ...,https://www.youtube.com/watch?v=7eyLjlJH9fw,A.K Music,https://www.youtube.com/channel/UCfHKS0-m8zC7U...,False,"9 Mar 2021, 17:26:05 IST"
11058,Imagine..,https://www.youtube.com/watch?v=6lKIh1aaCL0,[Anime] ZoneEdits,https://www.youtube.com/channel/UCWyUHSSq9sElv...,False,"9 Feb 2022, 18:24:42 IST"
7731,CSS Tutorial: Variables & Custom Properties | ...,https://www.youtube.com/watch?v=ghlm_94oR90,CodeWithHarry,https://www.youtube.com/channel/UCeVMnSShP_Ivi...,False,"21 Jun 2022, 08:13:28 IST"
6555,Convert Low-Res Graphic to High-Res in Photosh...,https://www.youtube.com/watch?v=I58HzUhQ3W8,PiXimperfect,https://www.youtube.com/channel/UCMrvLMUITAImC...,False,"26 Jul 2022, 18:16:34 IST"
12582,#PressPLAY | How I ACCIDENTALLY Remade Eurodan...,https://www.youtube.com/watch?v=aI19eFOw7qY,Vories,https://www.youtube.com/channel/UCmxmYvWPHQ2KO...,False,"21 Dec 2020, 12:56:10 IST"
11837,Arrays And Pointer Arithmetic In C: C Tutorial...,https://www.youtube.com/watch?v=RhLSYjDXB4Q,CodeWithHarry,https://www.youtube.com/channel/UCeVMnSShP_Ivi...,False,"28 Aug 2021, 22:49:08 IST"


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12624 entries, 0 to 12623
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Video Title      12624 non-null  object
 1   Video URL        12624 non-null  object
 2   Youtube Creator  10908 non-null  object
 3   YT Creator URL   10909 non-null  object
 4   Is Ad            12624 non-null  bool  
 5   Time             12624 non-null  object
dtypes: bool(1), object(5)
memory usage: 505.6+ KB


In [5]:
history = data.copy()
history.sample(3)

,Video Title,Video URL,Youtube Creator,YT Creator URL,Is Ad,Time
3532,6.3 Merge Operation in Binomial Heaps | Advanc...,https://www.youtube.com/watch?v=-82Wd64fxtw,Uzair Javed Akhtar,https://www.youtube.com/channel/UCOxgRflNQkRVT...,False,"19 Oct 2022, 07:01:20 IST"
2435,220624_EN_The glow up after a breakup_Vertical,https://www.youtube.com/watch?v=Cr5FtJWrgXI,NaN,NaN,True,"9 Dec 2022, 20:04:48 IST"
11554,How to Duplicate Diamonds in Minecraft,https://www.youtube.com/watch?v=hR1I_WnSlT0,Mementoe,https://www.youtube.com/channel/UCXhw8o3V525qX...,False,"25 Nov 2021, 17:35:02 IST"


transforming time col ( as there are some other string present with time in cell)

In [6]:
def ref_time(st):
    date_time_pattern = r'\b(?:0?[1-9]|[1-2][0-9]|3[01])\s(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)\s\d{4},\s\d{2}:\d{2}:\d{2}\s\w+\b'
    st = st.strip()
    matches = re.findall(date_time_pattern, st)

    if matches:
        extracted_date_time = matches[0]
        if extracted_date_time[0].isdigit() and extracted_date_time[1].isdigit():
            if int(extracted_date_time[0:2]) > 30:
                return extracted_date_time[1:]
        return extracted_date_time

    months = {
        'Jan' : 31, 'Feb' : [28, 29], 'Mar': 31, 'Apr': 30, 'May': 31, 'Jun': 30, 'Aug':31, ' Se': 30, 'Oct':31, 'Nov':30, 'Dec':31}

    if len(st) >= 26 and st[-26].isdigit():
        if st[-25].isdigit():
            cur = st[-24:-21]
            date = int(st[-26:-24])
            if cur != 'Feb':
                if  date <= months[cur]:
                        return st[-26:]
            else:
                yr = int(st[-18:-14])
                if (yr % 100 == 0 and yr % 4 == 0) or (yr % 100 != 0 and yr % 4 == 0):
                    if date <= months['Feb'][1]:
                        return st[-26:]
                else:
                    return st[-26:]
    return st[-25:]



history['Time'] = history['Time'].apply(ref_time)

In [7]:
history

,Video Title,Video URL,Youtube Creator,YT Creator URL,Is Ad,Time
0,How to Download your YouTube Data,https://www.youtube.com/watch?v=5CGgqJWh7wo,Buzz2day Tech,https://www.youtube.com/channel/UCSaA50dPTKM4X...,False,"23 Jul 2023, 19:20:17 IST"
1,"Farhan Akhtar ke bol, Farhan Akhtar ki awaaz m...",https://www.youtube.com/watch?v=fP-WXYk1wVc,NaN,NaN,True,"23 Jul 2023, 19:17:10 IST"
2,🎙️ From Syria to Revolutionising Men's Groomin...,https://www.youtube.com/watch?v=ekcgdD_q7bU,Sonny Sangha,https://www.youtube.com/channel/UCqeTj_QAnNlmt...,False,"23 Jul 2023, 19:15:09 IST"
3,MOVING TO DUBAI | Vlog #0,https://www.youtube.com/watch?v=hP1V5wehdEU,Sonny Sangha,https://www.youtube.com/channel/UCqeTj_QAnNlmt...,False,"23 Jul 2023, 19:15:00 IST"
4,Python Project to Scrape YouTube using YouTube...,https://www.youtube.com/watch?v=SwSbnmqk3zY,techTFQ,https://www.youtube.com/channel/UCnz-ZXXER4jOv...,False,"23 Jul 2023, 19:12:48 IST"
...,...,...,...,...,...,...
12619,10$ Drum Vs 1000$ Drum,https://www.youtube.com/watch?v=FnY_R0ZUGGM,Big Marvel,https://www.youtube.com/channel/UCgI8VzlDGsHCp...,False,"1 Aug 2020, 17:30:17 IST"
12620,Kokila Ben | First World Problems | Dialogue w...,https://www.youtube.com/watch?v=SjhBeIz1tDI,Yashraj Mukhate,https://www.youtube.com/channel/UCBZQAxIn4Mxc2...,False,"1 Aug 2020, 17:28:14 IST"
12621,"Mehul Choksi, Zakir Naik gave donation to Raji...",https://www.youtube.com/watch?v=9JbrEW1A0xY,Bharatiya Janata Party,https://www.youtube.com/channel/UCrwE8kVqtIUVU...,False,"1 Aug 2020, 16:21:46 IST"
12622,Prime Minister Narendra Modi's Mann Ki Baat wi...,https://www.youtube.com/watch?v=OgafNvdpGHI,Narendra Modi,https://www.youtube.com/channel/UC1NF71EwP41Vd...,False,"1 Aug 2020, 14:39:49 IST"


In [8]:
history['Time'].unique()

array(['23 Jul 2023, 19:20:17 IST', '23 Jul 2023, 19:17:10 IST',
       '23 Jul 2023, 19:15:09 IST', ..., '1 Aug 2020, 16:21:46 IST',
       '1 Aug 2020, 14:39:49 IST', '16 Jun 2020, 20:17:32 IST'],
      dtype=object)

In [9]:
history.sample(5)

,Video Title,Video URL,Youtube Creator,YT Creator URL,Is Ad,Time
5105,Build an OTT Platform | Masterclass,https://www.youtube.com/watch?v=-ieOIZN5vuY,NaN,NaN,True,"16 Sept 2022, 21:31:20 IST"
1296,6.6 Type 2 Grammar | Context free grammar | Ch...,https://www.youtube.com/watch?v=KPf1nGVLsns,KNOWLEDGE GATE,https://www.youtube.com/channel/UCA6yfpYhy5sWM...,False,"4 Apr 2023, 17:52:22 IST"
1879,Great photo taking in Phuket #photography,https://www.youtube.com/watch?v=HXjqdNivN-U,Semyon Rezchikov,https://www.youtube.com/channel/UCrCvnCuOe0zJ5...,False,"24 Feb 2023, 18:28:41 IST"
12361,Lime Soda Process of Water Softening,https://www.youtube.com/watch?v=7vZsAGSqVKY,Dr. Neha Patni,https://www.youtube.com/channel/UC_xCP3r27T85O...,False,"11 Apr 2021, 20:03:47 IST"
11511,Me Holding Up Tears In School:,https://www.youtube.com/watch?v=I4FoOoeNz5g,Viral TikToks,https://www.youtube.com/channel/UCqbyZbymyuvo1...,False,"25 Nov 2021, 17:45:39 IST"


In [10]:
history.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12624 entries, 0 to 12623
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Video Title      12624 non-null  object
 1   Video URL        12624 non-null  object
 2   Youtube Creator  10908 non-null  object
 3   YT Creator URL   10909 non-null  object
 4   Is Ad            12624 non-null  bool  
 5   Time             12624 non-null  object
dtypes: bool(1), object(5)
memory usage: 505.6+ KB


In [11]:
history['Time']

0        23 Jul 2023, 19:20:17 IST
1        23 Jul 2023, 19:17:10 IST
2        23 Jul 2023, 19:15:09 IST
3        23 Jul 2023, 19:15:00 IST
4        23 Jul 2023, 19:12:48 IST
                   ...            
12619     1 Aug 2020, 17:30:17 IST
12620     1 Aug 2020, 17:28:14 IST
12621     1 Aug 2020, 16:21:46 IST
12622     1 Aug 2020, 14:39:49 IST
12623    16 Jun 2020, 20:17:32 IST
Name: Time, Length: 12624, dtype: object

Most Watched Video

In [12]:
video_counts = history['Video URL'].value_counts()

most_common_video_url = video_counts.idxmax()
highest_count = video_counts.max()
most_common_video_rows = history[history['Video URL'] == most_common_video_url]

print("Most Common Video URL:", most_common_video_url)
print("Video Title: ", most_common_video_rows.iloc[0]['Video Title'])
print("Count:", highest_count)

Most Common Video URL: https://www.youtube.com/watch?v=xGPjonxrUYc
Video Title:  Hardwell Live at Ultra Music Festival Miami 2017
Count: 31


In [13]:
top_5_videos = video_counts.nlargest(5)

top_5_rows = history[history['Video URL'].isin(top_5_videos.index)]

unique_videos = top_5_rows.drop_duplicates(subset=['Video URL', 'Video Title', 'Youtube Creator'])

print(unique_videos[['Youtube Creator', 'Video Title']])

         Youtube Creator                                        Video Title
491             Hardwell   Hardwell Live at Ultra Music Festival Miami 2017
512             Hardwell   Hardwell LIVE at Ultra Music Festival Miami 2018
739      Nightblue Music  Warriyo - Mortals (feat. Laura Brehm) (ELPORT ...
7214          Capital FM  Taylor Swift - ‘...Ready For It?’ (Live At Cap...
8628  LeeandLie (AmaLee)  Sword Art Online - "Crossing Field" (REMIX) | ...


Most watched videos of yt creator

In [14]:
creator_counts = history['Youtube Creator'].value_counts()

most_watched_yt_creator = creator_counts.idxmax()
highest_count = creator_counts.max()

print("Most watched yt creator:", most_watched_yt_creator)
print("Video Count:", highest_count)

Most watched yt creator: TheMermaidscales
Video Count: 345


In [15]:
top_5_creators = creator_counts.nlargest(5)


print(top_5_creators)

TheMermaidscales    345
CodeWithHarry       337
Gate Smashers       168
Capital FM          119
Aditya Verma        109
Name: Youtube Creator, dtype: int64


In [79]:
top_videos_by_top_creators = history[history['Youtube Creator'].isin(top_5_creators.index)]

top_videos_by_creator_title = top_videos_by_top_creators.groupby(['Youtube Creator', 'Video Title']).size()

top_videos_by_creator_title = top_videos_by_creator_title.reset_index(name='Watch Count')
top_5_videos_by_creator = top_videos_by_creator_title.groupby('Youtube Creator').apply(lambda x: x.nlargest(5, 'Watch Count')).reset_index(drop=True)
print(top_5_videos_by_creator)

     Youtube Creator                                        Video Title  \
0       Aditya Verma                           4 01knapsack memoization   
1       Aditya Verma                   delete middle element of a stack   
2       Aditya Verma                              13 unbounded knapsack   
3       Aditya Verma     15  coin change problem maximum number of ways   
4       Aditya Verma           19  longest common subsequence recursive   
5         Capital FM  taylor swift  ready for it live at capitals ji...   
6         Capital FM  zara larsson  lush life live at the summertime...   
7         Capital FM  ariana grande  bang bang live at capitals summ...   
8         Capital FM  taylor swift  you need to calm down live at ca...   
9         Capital FM  zara larsson  lush life  live at capitals jing...   
10     CodeWithHarry  component structure of our newsmonkey react ap...   
11     CodeWithHarry  creating our first react app using createreact...   
12     CodeWithHarry  fix

In [80]:
from nltk.tokenize import word_tokenize
# import nltk
from collections import Counter
from nltk.corpus import stopwords

In [81]:
# nltk.download('punkt')
# nltk.download('stopwords')

In [82]:

# Convert the text column to lowercase
history['Video Title'] = history['Video Title'].str.lower()

keywords = history['Video Title'].apply(word_tokenize)

stop_words = set(stopwords.words('english'))

In [83]:
keywords

0                 [how, to, download, your, youtube, data]
1        [farhan, akhtar, ke, bol, farhan, akhtar, ki, ...
2        [from, syria, to, revolutionising, mens, groom...
3                             [moving, to, dubai, vlog, 0]
4        [python, project, to, scrape, youtube, using, ...
                               ...                        
12619                           [10, drum, vs, 1000, drum]
12620    [kokila, ben, first, world, problems, dialogue...
12621    [mehul, choksi, zakir, naik, gave, donation, t...
12622    [prime, minister, narendra, modis, mann, ki, b...
12623    [2020, the, worst, year, depression, helpline,...
Name: Video Title, Length: 12624, dtype: object

In [84]:
all_keywords = [token for sublist in keywords for token in sublist if token not in stop_words]

cnt = Counter(all_keywords)

In [87]:
keywords_df = pd.DataFrame(cnt.items(), columns=['Keyword', 'Count'])


keywords_df = keywords_df.sort_values(by='Count', ascending=False)

print("Top 5 recurring topics in title: ")
print(keywords_df.head(5))

Top 5 recurring topics in title: 
     Keyword  Count
107   shorts   1613
324    hindi    496
108    anime    444
96      live    432
135  english    414
